打包成压缩文件

In [ ]:
!zip -r Ultralight-Digital-Human3.zip /mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human

检查依赖工具

In [ ]:
import sys, os, textwrap, subprocess, torch

!nvidia-smi
print("Python version:", sys.version)

# 建议先回到 /content
# %cd /content

# # 克隆 Ultralight-Digital-Human 仓库
# !git clone https://github.com/anliyuan/Ultralight-Digital-Human.git
# %cd /content/Ultralight-Digital-Human
# !ls



安装依赖

In [3]:
%cd /mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human

# 不再强行装旧版 torch / numpy，直接用系统已有版本
import torch, numpy as np
print("Torch version:", torch.__version__)
print("Numpy version:", np.__version__)

# 安装其余依赖，全部用“不要锁死版本”的形式
!pip install -U pip
!pip install mkl opencv-python transformers soundfile librosa onnxruntime tqdm einops timm wget matplotlib



/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human
Torch version: 2.8.0+cu128
Numpy version: 1.26.4
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 738.0 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.0/195.0 MB 1.2 MB/s  0:02:490:00:0100:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 1.3 MB/s  0:00:13m0:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 MB 1.2 MB/s  0:01:00m0:00:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 1.4 MB/s  0:00:02 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

设定初始路径

In [1]:
PROJECT_ROOT = "/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human"
%cd $PROJECT_ROOT
!ls

/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human
build_stream_data.py   inference.py	   syncnet.py
checkpoint_wenet       landmarks	   test_16k.wav
data_dir	       offline_demo.py	   test_audio.pcm
datasetsss.py	       pth2onnx.py	   test_video.mp4
data_utils	       __pycache__	   train.py
demo		       README.md	   unet.py
dh_stream_service.zip  result_test.mp4	   wenet_result_final.mp4
dihuman_run.py	       stream_data	   wenet_result_silent.mp4
EAS_service	       syncnet_ckpt
full_body_img	       syncnet_ckpt_wenet


删除之前生成的冗余文件

In [13]:
# !rm -rf /content/drive/MyDrive/Ultralight-Digital-Human/data_dir/full_body_img
# !rm -rf /content/drive/MyDrive/Ultralight-Digital-Human/data_dir/landmarks
# !rm -rf /content/drive/MyDrive/Ultralight-Digital-Human/data_dir/aud.wav
# !rm -rf /content/drive/MyDrive/Ultralight-Digital-Human/data_dir/aud_hu.npy
# !rm -rf /mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human/content


确认 encoder.onnx 已经在 data_utils/

In [13]:
%cd $PROJECT_ROOT
!pip install gdown

# README 提供的 wenet encoder.onnx
ENCODER_ID = "1e4Z9zS053JEWl6Mj3W9Lbc9GDtzHIg6b"
!gdown "https://drive.google.com/uc?id={ENCODER_ID}" -O data_utils/encoder.onnx
!ls data_utils


/content/drive/MyDrive/Ultralight-Digital-Human
Downloading...
From (original): https://drive.google.com/uc?id=1e4Z9zS053JEWl6Mj3W9Lbc9GDtzHIg6b
From (redirected): https://drive.google.com/uc?id=1e4Z9zS053JEWl6Mj3W9Lbc9GDtzHIg6b&confirm=t&uuid=e614d05c-97d0-4a85-8a28-fd44a29e4ed6
To: /content/drive/MyDrive/Ultralight-Digital-Human/data_utils/encoder.onnx
100% 115M/115M [00:02<00:00, 46.4MB/s]
base_module.py		      FeaturePipeline.py  __pycache__
checkpoint_epoch_335.pth.tar  get_landmark.py	  scrfd_2.5g_kps.onnx
conf			      hubert.py		  wenet
decode_engine_V4.yaml	      mean_face.txt	  wenet_infer.py
detect_face.py		      pfld_mobileone.py
encoder.onnx		      process.py


上传训练视频 + 转成 20fps

In [14]:
%cd $PROJECT_ROOT

VIDEO_IN = "/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human/data_dir/test.mp4"          # 训练视频文件名
VIDEO_20 = "/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human/data_dir/test_20fps.mp4"    # 转码后的文件

# 看原始信息（可选）
!ffmpeg -i $VIDEO_IN

# 转成 20fps（如果本来就是 20fps，这步可以跳过，直接用原视频）
!ffmpeg -y -i $VIDEO_IN -r 20 $VIDEO_20





/content/drive/MyDrive/Ultralight-Digital-Human
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libv

预处理（抽帧 + 关键点 + wenet 音频特征）


In [17]:
# 再训练主模型，直接保存到云盘
%cd $PROJECT_ROOT/data_utils
VIDEO_IN = "/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human/data_dir/test.mp4"          # 训练视频文件名
VIDEO_20 = "/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human/data_dir/test_20fps.mp4"    # 转码后的文件
ASR_MODE = "wenet"
VIDEO_PATH = VIDEO_20

!python process.py $VIDEO_PATH --asr $ASR_MODE


/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human/data_utils
[INFO] ===== extract audio from /mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human/data_dir/test_20fps.mp4 to /mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human/data_dir/aud.wav =====
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --en

训练 SyncNet（用 wenet）

In [18]:
%cd $PROJECT_ROOT

SYNC_SAVE_DIR = "./syncnet_ckpt_wenet"
!mkdir -p $SYNC_SAVE_DIR

!python syncnet.py \
    --save_dir $SYNC_SAVE_DIR \
    --dataset_dir ./data_dir \
    --asr wenet


/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human
/usr/local/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
0 0.009498539380729198
1 0.00571874063462019
2 0.004487020429223776
3 0.00344818364828825
4 0.0026781552005559206
5 0.002186375204473734
6 0.0019503781804814935
7 0.0016864899080246687
8 0.0012974105775356293
9 0.0013028610264882445
10 0.001107160933315754
11 0.0009404565207660198
12 0.0008399059297516942
13 0.0008139574783854187
14 0.0006574345752596855
15 0.000715642177965492
16 0.0005147591000422835
17 0.0005117449909448624
18 0.0005382719682529569
19 0.00039776699850335717
20 0.00039221058250404894
21 0.0004317555285524577
22 0.00034726838930509984
23 0.0002955798991024494
24 0.00035611455678008497
25 0.000310

训练数字人主模型（UNet，用 wenet）

In [19]:
# 先看有哪些 checkpoint：
%cd $PROJECT_ROOT
!ls syncnet_ckpt_wenet


/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human
0.pth	13.pth	17.pth	20.pth	24.pth	28.pth	31.pth	35.pth	39.pth	6.pth
10.pth	14.pth	18.pth	21.pth	25.pth	29.pth	32.pth	36.pth	3.pth	7.pth
11.pth	15.pth	19.pth	22.pth	26.pth	2.pth	33.pth	37.pth	4.pth	8.pth
12.pth	16.pth	1.pth	23.pth	27.pth	30.pth	34.pth	38.pth	5.pth	9.pth


In [26]:
%cd $PROJECT_ROOT

SAVE_DIR = "./checkpoint_wenet"
!mkdir -p $SAVE_DIR

!python train.py \
    --dataset_dir ./data_dir \
    --save_dir $SAVE_DIR \
    --asr wenet \
    --use_syncnet \
    --syncnet_checkpoint syncnet_ckpt_wenet/39.pth   #选39.pth



/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human
/usr/local/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/usr/local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoc

准备测试音频（用 wenet 提特征）

In [30]:
%cd $PROJECT_ROOT/data_utils

TEST_WAV = "mkas.wav"  # 换成真实文件名

# 如果采样率不是 16k，可以先转一份：
!ffmpeg -y -i $TEST_WAV -ar 16000 test_16k.wav
TEST_WAV = "test_16k.wav"

# 用 wenet 脚本提取特征
!python wenet_infer.py $TEST_WAV

!ls


/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human/data_utils
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable

推理（用 wenet 模型 + wenet 特征）

In [35]:
%cd $PROJECT_ROOT

ASR_MODE   = "wenet"
DATASET    = "./data_dir"              # 训练时用的视频 data_dir
AUDIO_FEAT = "data_utils/test_16k_wenet.npy"      # 上一步生成的 npy 文件
CKPT       = "checkpoint_wenet/45.pth" # 换成你想用的模型
OUT_VIDEO  = "wenet_result_silent.mp4"

!python inference.py \
    --asr $ASR_MODE \
    --dataset $DATASET \
    --audio_feat $AUDIO_FEAT \
    --save_path $OUT_VIDEO \
    --checkpoint $CKPT


/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human
/usr/local/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


用 ffmpeg 合成有声音的视频

In [36]:
%cd $PROJECT_ROOT

VIDEO_SILENT = "wenet_result_silent.mp4"
AUDIO_WAV    = TEST_WAV             # 即 test_16k.wav
FINAL_OUT    = "wenet_result_final.mp4"

!ffmpeg -y -i $VIDEO_SILENT -i $AUDIO_WAV -c:v libx264 -c:a aac $FINAL_OUT


/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame

拷贝流式推理需要的对应文件

In [4]:

%cd $PROJECT_ROOT
!pip install onnx
CKPT       = "checkpoint_wenet/45.pth"
DATASET    = "./data_dir"

!python build_stream_data.py \
  --weights $CKPT \
  --data_dir $DATASET \
  --mode "wenet" \
  --out_dir "stream_data"



/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 1.3 MB/s  0:00:14m0:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 1.2 MB/s  0:00:03 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [onnx]1/2 [onnx]
/usr/local/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
         Build Stream Data
[✔] 已复制 5291 张图片 → stream_data/img_inference
[✔] 已复制 5291 个 landmarks → stream_data/lms_inference
[INFO] Using device: cuda
[INFO] Model loaded from: checkpoint_wenet/45.pth
/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human/build_stream_data.py:40: DeprecationWarning: You are usi

删掉坏文件

In [57]:
!rm test_video.mp4 test_audio.pcm


rm: 无法删除 'test_video.mp4': 没有那个文件或目录
rm: 无法删除 'test_audio.pcm': 没有那个文件或目录


运行流式推理

In [58]:
# !pip install kaldi_native_fbank
!python dihuman_run.py --data_path "./stream_data/"


>>> step1: reading wav...
    wav shape: (224955,) sample_rate: 16000
>>> step2: init processor...
[INFO] found 5291 images, 5291 lms, using 5291 frames
[INFO] loading frame 0/300
[INFO] loading frame 50/300
[INFO] loading frame 100/300
[INFO] loading frame 150/300
[INFO] loading frame 200/300
[INFO] loading frame 250/300
/usr/local/lib/python3.11/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:123: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
    loaded 300 frames
>>> step3: audio_stream_len = 224955 num_chunks = 1406
>>> step4: release video, total frames: 283
>>> step5: write pcm, total samples: (224960,)


检查生成的文件是否完好

In [59]:
!ls -lh test_video.mp4
!ffprobe test_video.mp4



-rw-r--r-- 1 root root 8.4M 12月  1 20:18 test_video.mp4
ffprobe version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2007-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --en

In [60]:
# !rm result_test.mp4

合成音频和视频

In [61]:
!ffmpeg -y -i test_video.mp4 -f s16le -ar 16000 -ac 1 -i test_audio.pcm \
       -c:v libx264 -c:a aac result_test.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

查看结果视频的大小检查是否完好（正常应该是几m）

In [62]:
ls -lh result_test.mp4


-rw-r--r-- 1 root root 5.6M 12月  1 20:18 result_test.mp4


把数据文件复制到最终的文件夹里

In [5]:
# !cp -r stream_data EAS_service

打包数据文件

In [ ]:
# !zip -r dh_stream_service.zip EAS_service/


git管理

In [ ]:
%cd /mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human/EAS_service
!ls
!git init
!git branch -M main
!git remote add origin git@github.com:mirror1717/AI-Interviewer-THG.git
!git branch -M main




git提交修改

In [27]:
!git add .
!git status
!git restore --staged .ipynb_checkpoints/frame_000-checkpoint.jpg
!git restore --staged

位于分支 main
您的分支与上游分支 'origin/main' 一致。

要提交的变更：
  （使用 "git restore --staged <文件>..." 以取消暂存）
	新文件：   .ipynb_checkpoints/digital_human-checkpoint.ipynb
	新文件：   .ipynb_checkpoints/frame_000-checkpoint.jpg
	新文件：   digital_human.ipynb
	新文件：   frame_000.jpg
	新文件：   test_http_audio.pcm



In [ ]:
!git commit -m "Initial commit: add EAS service core files"
!git -c "core.sshCommand=ssh -o StrictHostKeyChecking=no" push -u origin main



本地测试流程

In [20]:
%cd /mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human/EAS_service
!python -m uvicorn server:app --host 0.0.0.0 --port 8000


/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human/EAS_service
INFO:     Started server process [1843]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
^C
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1843]


In [21]:
!curl http://127.0.0.1:8000/health


curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: 连接被拒绝


测试一帧（客户端）

In [ ]:
import soundfile as sf
import numpy as np
import base64
import requests

# ===== 1. 读本地 wav =====
wav_path = "/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human/test_16k.wav"   # 换成你自己的路径
audio, sr = sf.read(wav_path)   # audio: float32/64, [-1, 1] or 任意

print("wav shape:", audio.shape, "sr:", sr)

# 如果是双声道，取一条
if audio.ndim > 1:
    audio = audio[:, 0]

# ===== 2. 转成 int16 PCM =====
audio = audio.astype(np.float32)

# 一般 wav 是 [-1, 1]，先放大到 int16 范围
if audio.max() <= 1.0 and audio.min() >= -1.0:
    audio = audio * 32767.0

pcm_int16 = audio.astype(np.int16)   # 关键：你的 DiHumanProcessor 要的就是这个

# 取一帧 10ms（160 个采样）
chunk = pcm_int16[:160]   # np.int16, shape [160]

# ===== 3. int16 → bytes → base64 字符串 =====
pcm_bytes = chunk.tobytes()                       # 160 * 2 字节
audio_b64 = base64.b64encode(pcm_bytes).decode("ascii")  # 发给接口的就是这个字符串

# ===== 4. 发送到你的 FastAPI/EAS =====
url = "http://127.0.0.1:8000/stream_step"# 本地测试; 部署到EAS后换成 EAS_ENDPOINT + /stream_step

payload = {
    "session_id": "test-session-1",
    "audio_chunk": audio_b64,
    "reset": False,
}

resp = requests.post(url, json=payload)
print("status:", resp.status_code)
print("resp json:", resp.json())


把图片json转换成图片（客户端）

In [24]:
import base64
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt

# 假设你刚刚的响应是：
# resp_json = {'frame': '...很长很长的base64...'}
resp_json = resp.json()

b64_str = resp_json['frame']

# 1）base64 解码成二进制
img_bytes = base64.b64decode(b64_str)

# 2）用 PIL 打开
img = Image.open(BytesIO(img_bytes))

# 3）保存为jpg图片
img.save("frame_000.jpg", quality=95)


多帧流式调用

In [25]:
import soundfile as sf
import numpy as np
import base64
import requests
import math
import cv2
from scipy.io import wavfile

wav_path = "/mnt/workspace/content/drive/MyDrive/Ultralight-Digital-Human/test_16k.wav"
audio, sr = sf.read(wav_path)

if audio.ndim > 1:
    audio = audio[:, 0]

audio = audio.astype(np.float32)
if audio.max() <= 1.0 and audio.min() >= -1.0:
    audio = audio * 32767.0
pcm_int16 = audio.astype(np.int16)

print("audio len:", len(pcm_int16), "sr:", sr)

# FastAPI / EAS 地址
url = "http://127.0.0.1:8000/stream_step"   # EAS 上改成 http://.../api/predict/xxx/stream_step
session_id = "test-session-1"

# 视频帧和音频片段缓存
frames = []
audio_chunks = []

# 先告诉后端 reset 一下（清空内部状态）
def call_step(chunk_int16, reset=False):
    # chunk_int16: np.int16 一维数组
    pcm_bytes = chunk_int16.tobytes()
    audio_b64 = base64.b64encode(pcm_bytes).decode("ascii")
    payload = {
        "session_id": session_id,
        "audio_chunk": audio_b64,
        "reset": reset,
    }
    r = requests.post(url, json=payload)
    r.raise_for_status()
    return r.json()

# 第一次调用，顺便 reset
first_chunk = pcm_int16[:160] if len(pcm_int16) >= 160 else pcm_int16
resp0 = call_step(first_chunk, reset=True)
print("first resp:", resp0)

# 把第一次的结果也存下来
audio_chunks.append(base64.b64decode(resp0["audio"]))
if resp0["check_img"] == 1 and resp0["frame"] is not None:
    img_bytes = base64.b64decode(resp0["frame"])
    img_arr = np.frombuffer(img_bytes, dtype=np.uint8)
    frame = cv2.imdecode(img_arr, cv2.IMREAD_COLOR)
    frames.append(frame)

# 后续流式发送
num_chunks = math.ceil(len(pcm_int16) / 160)
for i in range(1, num_chunks):
    start = i * 160
    end = min((i + 1) * 160, len(pcm_int16))
    chunk = pcm_int16[start:end]

    # 不足 160 的补零，和你原来的逻辑一致
    if len(chunk) < 160:
        padding = np.zeros(160 - len(chunk), dtype=np.int16)
        chunk = np.concatenate([chunk, padding])

    resp = call_step(chunk, reset=False)

    audio_chunks.append(base64.b64decode(resp["audio"]))

    if resp["check_img"] == 1 and resp["frame"] is not None:
        img_bytes = base64.b64decode(resp["frame"])
        img_arr = np.frombuffer(img_bytes, dtype=np.uint8)
        frame = cv2.imdecode(img_arr, cv2.IMREAD_COLOR)
        frames.append(frame)

print("得到视频帧数：", len(frames))
print("得到音频片段数：", len(audio_chunks))

# 把帧写成视频（可选）
if frames:
    h, w, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter("test_http_video.mp4", fourcc, 20, (w, h))
    for f in frames:
        if f.shape[:2] != (h, w):
            f = cv2.resize(f, (w, h))
        writer.write(f)
    writer.release()

# 把音频写成 pcm（注意：这里写的是 16kHz，和你原来的逻辑一致）
audio_bytes_all = b"".join(audio_chunks)
audio_np = np.frombuffer(audio_bytes_all, dtype=np.int16)
wavfile.write("test_http_audio.pcm", 16000, audio_np)
print("最终音频采样点数：", audio_np.shape)


audio len: 224955 sr: 16000
first resp: {'frame': '...', 'audio': 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA=', 'check_img': 1}
得到视频帧数： 283
得到音频片段数： 1406
最终音频采样点数： (224960,)


合成音频和视频

In [26]:
!ffmpeg -y -i test_http_video.mp4 -f s16le -ar 16000 -ac 1 -i test_http_audio.pcm \
       -c:v libx264 -c:a aac result_http_test.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

测试EAS服务

In [7]:
import requests
import base64
import json
import numpy as np

# ===============================
# 1. 你的 EAS 服务配置
# ===============================
EAS_ENDPOINT = "http://1577983101063836.cn-hangzhou.pai-eas.aliyuncs.com/api/predict/digital_human_thg2"
EAS_TOKEN = "ZDViOGY5YjlmZDU3NjM0MWNlZTBhYWYzMGY0NjNhZGE5M2ZjZTYwZg=="   # <---- 你需要手动填写

# FastAPI 文档地址（可测试服务已是否健康）
DOCS_URL = f"{EAS_ENDPOINT}/docs"

headers = {
    "Authorization": EAS_TOKEN,
    "Content-Type": "application/json"
}

# ===============================
# 2. 测试基础连通性：访问 /docs
# ===============================
print("Testing /docs ...")

resp = requests.get(DOCS_URL, headers=headers)

print("Status Code:", resp.status_code)
print("First 300 chars of response:")
print(resp.text[:300])


# ===============================
# 3. 测试你的 API（例子：/stream_step）
# ===============================
# 如果你的服务有其他路径，请把 /stream_step 改成你实际的 API
api_url = f"{EAS_ENDPOINT}/stream_step"

# 构造一个假音频的 base64 数据（真实使用时替换）
# 1. 生成一小段 16bit PCM 假音频（比如 160 个采样点，全 0 静音）
num_samples = 160
pcm_int16 = np.zeros(num_samples, dtype=np.int16)  # [0, 0, 0, ...]
pcm_bytes = pcm_int16.tobytes()                    # 长度 = 160 * 2 = 320 字节，肯定是 2 的倍数

# 2. base64 编码（注意是对 bytes 编码）
audio_chunk_b64 = base64.b64encode(pcm_bytes).decode("ascii")
# dummy_audio ="test_16k.wav"

data = {
    "session_id": "test_123",
    "audio_chunk": dummy_audio
}

print("\nTesting /stream_step ...")

resp2 = requests.post(api_url, headers=headers, json=data)

print("Status Code:", resp2.status_code)
try:
    print("JSON Response:", resp2.json())
except Exception:
    print("Raw Response:", resp2.text)


# ===============================
# 4. 如果需要持续对话，可循环发送请求
# ===============================




Testing /docs ...
Status Code: 200
First 300 chars of response:

    <!DOCTYPE html>
    <html>
    <head>
    <link type="text/css" rel="stylesheet" href="https://cdn.jsdelivr.net/npm/swagger-ui-dist@5/swagger-ui.css">
    <link rel="shortcut icon" href="https://fastapi.tiangolo.com/img/favicon.png">
    <title>FastAPI - Swagger UI</title>
    </head>
    <body

Testing /stream_step ...
Status Code: 500
Raw Response: Internal Server Error
